In [1]:
import pandas as pd
import numpy as np
import re

In [13]:
council_housing = pd.read_csv(r'../../data/processed/council_housing_cleaned.csv')

In [14]:
council_housing 

,Local authority,Right_to_Buy_total_number_of_dwellings,Social_Homebuy_number_of_dwellings,Other_sales_to_sitting_tenants_number_of_dwellings,Other_sales_number_of_dwellings,Transfers_to_PRPs,Sales_of_Shared_Ownership_number_of_dwellings,Financial_Year
0,County Durham,0.0,0,0,0,0,0,2023
1,Darlington,20.0,0,0,0,0,0,2023
2,Gateshead,117.0,0,0,0,0,7,2023
3,Hartlepool,0.0,0,0,0,0,0,2023
4,Middlesbrough,0.0,0,0,0,0,0,2023
...,...,...,...,...,...,...,...,...
3331,Malvern Hills,0.0,0.0,0.0,0.0,0.0,NaN,2014
3332,Redditch,41.0,0.0,0.0,0.0,0.0,NaN,2014
3333,Worcester,0.0,0.0,0.0,0.0,0.0,NaN,2014
3334,Wychavon,0.0,0.0,0.0,0.0,0.0,NaN,2014
